# 1. Install All Required Packages

In [ ]:
!pip -q install langchain langchain_community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip -q install datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate transformers

In [ ]:
!pip -q install unstructured
!pip install pinecone

In [ ]:
!pip install tokenizers

In [ ]:
!pip install xformers

# 2. Import all the Required Libraries

In [ ]:
import os
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import torch

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# 3. Pass the URLs and extract the data from these URLs

In [ ]:
URLs = [
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.databricks.com/blog/mpt-7b',
    'https://lmsys.org/blog/2023-03-30-vicuna/',
    'https://stability.ai/news/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models'
]

In [ ]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [ ]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Sitemap\n\nOpen in app\n\nSign in\n\nWrite\n\nSearch\n\nSign in\n\nGoPenAI\n\nGoPenAI\n\nWhere the ChatGPT community comes together to share insights and stories.\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nAndrew Lukyanenko\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, the

In [ ]:
len(data)

4

# 4. Split the Text into Chunks


In [ ]:
text_splitter = CharacterTextSplitter(separator = '\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

88

In [ ]:
docs

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Sitemap\nOpen in app\nSign in\nWrite\nSearch\nSign in\nGoPenAI\nGoPenAI\nWhere the ChatGPT community comes together to share insights and stories.\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nAndrew Lukyanenko\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link'),
 Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Llama 2: one of the best open source models\nAndrew Lukyanenko\nAndrew Lukyanenko\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion para

In [ ]:
docs[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Sitemap\nOpen in app\nSign in\nWrite\nSearch\nSign in\nGoPenAI\nGoPenAI\nWhere the ChatGPT community comes together to share insights and stories.\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nAndrew Lukyanenko\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link')

In [ ]:
docs[1]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Llama 2: one of the best open source models\nAndrew Lukyanenko\nAndrew Lukyanenko\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhancements for Llama 2-Chat to support the community in further developing and responsibly handling LLMs.\nPretraining')

# 5. Download the Hugging Face Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

768

In [ ]:
query_result

[0.026249706745147705,
 0.013395607471466064,
 -0.004533133003860712,
 -0.021791422739624977,
 0.05455183610320091,
 -0.004966470878571272,
 0.006655557081103325,
 0.03062625601887703,
 -0.005762824323028326,
 -0.004562034271657467,
 -0.003313301596790552,
 -0.0484962984919548,
 -0.011364065110683441,
 0.03507744148373604,
 0.09309469908475876,
 -0.0866873636841774,
 0.051086537539958954,
 0.009886160492897034,
 -0.06356934458017349,
 -0.008550204336643219,
 0.007054392248392105,
 -0.003862351179122925,
 0.024744288995862007,
 0.04288491606712341,
 0.035094160586595535,
 -0.02984827198088169,
 0.010252618230879307,
 0.02234490029513836,
 0.020889993757009506,
 0.00949221383780241,
 -0.03304434195160866,
 -0.012284105643630028,
 0.05352889373898506,
 0.02542915940284729,
 2.022177341132192e-06,
 -0.034191031008958817,
 0.009610011242330074,
 -0.016484549269080162,
 0.005609536077827215,
 -0.004250032361596823,
 -0.022801222279667854,
 0.04035475477576256,
 0.0030519880820065737,
 0.0313

# 6. Convert the Text Chunks into Embeddings and Create a Knowledge Base


In [ ]:
import pinecone
import os

os.environ["PINECONE_API_KEY"] = "pcsk_4MqwfK_Ew2McJ1VDDHvhZT22gqPHN4xUYYnGiso7M8HS9qPxxncQ1dTPNR2aw23PRr6nS8"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

In [ ]:
from pinecone import Pinecone

pc = Pinecone(
      api_key=os.environ.get("PINECONE_API_KEY")
)

In [ ]:
index_name = "webbot"
index = pc.Index(index_name)

In [ ]:
vectors = []
for i, doc in enumerate(docs):
    vector = embeddings.embed_query(doc.page_content)  # returns list of floats
    metadata = {
        "text": doc.page_content
    }
    vectors.append((f"doc{i+1}", vector, metadata))

In [ ]:
vectors[0]

('doc1',
 [0.04748029634356499,
  -0.0002539554552640766,
  -0.02391093410551548,
  0.009297857992351055,
  -0.03091498464345932,
  -0.01700248196721077,
  -0.0019030976109206676,
  0.03338008001446724,
  0.08812624961137772,
  -0.028129732236266136,
  -0.0454791821539402,
  0.0016761843580752611,
  0.01235994789749384,
  0.063628651201725,
  0.018520478159189224,
  -0.0726858600974083,
  0.006102553103119135,
  0.0022683730348944664,
  -0.019748851656913757,
  0.021218663081526756,
  0.016499044373631477,
  -0.010746587999165058,
  -0.010406003333628178,
  0.05771421268582344,
  0.014165610074996948,
  -0.04668352007865906,
  -0.029189258813858032,
  -0.007291784975677729,
  -0.016625871881842613,
  -0.0328165739774704,
  0.01893806830048561,
  0.011091778986155987,
  -0.02350306138396263,
  0.02522628754377365,
  2.4354280867555644e-06,
  -0.04151463136076927,
  -0.0024175895377993584,
  -0.011996684595942497,
  -0.03446201980113983,
  0.04475066065788269,
  0.03804631903767586,
  0.

In [ ]:
index.upsert(vectors=vectors)

{'upserted_count': 88}

# 7. Create a Large Language Model Wrapper

In [ ]:
notebook_login()

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens=512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

/tmp/ipython-input-899405379.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [ ]:
llm.invoke("Please provide a concise summary of the Book Harry Potter")

/tmp/ipython-input-1177971522.py:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")


'Please provide a concise summary of the Book Harry Potter and the Philosopher\'s Stone by J.K. Rowling.\n\nHarry Potter is an orphan who lives with his cruel and neglectful uncle, aunt, and spoiled cousin, Dudley Dursley. On his eleventh birthday, Harry discovers that he is a wizard, and he begins attending Hogwarts School of Witchcraft and Wizardry. He makes friends with Ron Weasley and Hermione Granger, and together they become entangled in a mystery surrounding the powerful Sorcerer\'s Stone.\n\nHarry learns that the Sorcerer\'s Stone is hidden somewhere within Hogwarts, and that the school\'s headmaster, Professor Dumbledore, is trying to protect it from the evil wizard, Lord Voldemort, who killed Harry\'s parents and seeks to return to power. Harry and his friends must navigate the challenges of school and friendship while also trying to uncover the truth about the Sorcerer\'s Stone and prevent Voldemort from obtaining it.\n\nThroughout the book, Harry faces obstacles and danger,

# Retrieve documents for Vector DB and Test


In [ ]:
def retrieve_documents(query: str, top_k: int = 3):
    """Embed query, search Pinecone, and return text content"""
    query_vector = embeddings.embed_query(query)
    results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)

    docs = []
    for match in results["matches"]:
        docs.append({
            "text": match["metadata"].get("text", ""),
            "score": match["score"]
        })
    return docs

In [ ]:
def retrieval_qa(question: str, top_k: int = 3):
    # Step 1: Retrieve top documents
    retrieved_docs = retrieve_documents(question, top_k)

    # Step 2: Combine retrieved text into a single context
    context = "\n\n".join([doc["text"] for doc in retrieved_docs])

    # Step 3: Construct the prompt (like chain_type="stuff")
    prompt = f"""
You are a helpful assistant. Use the context below to answer the question.
If the answer isn't clear, say so. Be concise and accurate.

Context:
{context}

Question: {question}

Answer:
"""

    # Step 4: Query the LLM
    response = llm.invoke(prompt)

    # Step 5: Handle both possible return types (string or object)
    if isinstance(response, str):
        answer_text = response.strip()
    else:
        # if it's a ChatMessage / LLMResult-like object
        answer_text = getattr(response, "content", str(response)).strip()

    # Step 6: Return result
    return {
        "answer": answer_text,
        "retrieved_texts": [doc["text"][:200] + "..." for doc in retrieved_docs],
        "scores": [doc["score"] for doc in retrieved_docs]
    }

In [ ]:
query = "How does Llama 2 outperform other models?"
result = retrieval_qa(query)

print("Answer:\n", result["answer"])
print("\nTop Retrieved Chunks:")
for i, snippet in enumerate(result["retrieved_texts"], 1):
    print(f"{i}. ({result['scores'][i-1]:.3f}) {snippet}")

Answer:
 You are a helpful assistant. Use the context below to answer the question.
If the answer isn't clear, say so. Be concise and accurate.

Context:
Llama 2 models with 7 billion and 30 billion parameters outdo MPT models of similar size in all categories except code benchmarks.
In comparison with Falcon models, Llama 2’s 7 billion and 34 billion parameter models outperform the 7 billion and 40 billion parameter Falcon models in all benchmark categories.
Moreover, the Llama 2 70B model surpasses all open-source models.
Comparatively, the Llama 2 70B model performs similarly to the closed-source GPT-3.5 (OpenAI, 2023) on the MMLU and GSM8K benchmarks but shows a significant deficit on coding benchmarks. It matches or exceeds the performance of PaLM (540 billion parameters) on nearly all benchmarks. However, there remains a substantial performance gap between the Llama 2 70B model and both GPT-4 and PaLM-2-L.
Fine-tuning
Supervised Fine-Tuning (SFT)

When scoring by preference ratin

In [ ]:
query = "How good is Vicuna?"
result = retrieval_qa(query)

print("Answer:\n", result["answer"])
print("\nTop Retrieved Chunks:")
for i, snippet in enumerate(result["retrieved_texts"], 1):
    print(f"{i}. ({result['scores'][i-1]:.3f}) {snippet}")

Answer:
 You are a helpful assistant. Use the context below to answer the question.
If the answer isn't clear, say so. Be concise and accurate.

Context:
LMSYS ORG
ProjectsBlogAboutDonationsChatbot Arena (graduated)
Vicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality
by: The Vicuna Team, Mar 30, 2023
We introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.
Vicuna (generated by stable diffusion 2.1)
According to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.
How Good is Vicuna?

Vicuna (generate

In [ ]:
query = "What is stableLM?"
result = retrieval_qa(query)

print("Answer:\n", result["answer"])
print("\nTop Retrieved Chunks:")
for i, snippet in enumerate(result["retrieved_texts"], 1):
    print(f"{i}. ({result['scores'][i-1]:.3f}) {snippet}")

Answer:
 You are a helpful assistant. Use the context below to answer the question.
If the answer isn't clear, say so. Be concise and accurate.

Context:
Stability AI Launches the First of its Stable LM Suite of Language Models
Product
Apr 19
Written By Joshua Lopez
Today, Stability AI released a new open source language model, Stable LM. The Alpha version of the model is available in 3 billion and 7 billion parameters, with 15 billion to 65 billion parameter models to follow. Developers can freely inspect, use, and adapt our Stable LM base models for commercial or research purposes, subject to the terms of the CC BY-SA-4.0 license.
In 2022, Stability AI drove the public release of Stable Diffusion, a revolutionary image model representing a transparent, open, and scalable alternative to proprietary AI. With the launch of the Stable LM suite of models, Stability AI is continuing to make foundational AI technology accessible to all. Our Stable LM models can generate text and code and wi